In [5]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from prefect import flow
# Use the proper import for deployments if needed (depending on your version)
# For example, in Prefect 3, the flow.from_source method is asynchronous.

SOURCE_REPO = "https://github.com/prefecthq/demos.git"

async def register_deployment():
    # Await the from_source call to load the flow
    loaded_flow = await flow.from_source(
        source=SOURCE_REPO,
        entrypoint="my_workflow.py:show_stars",  # Adjust to your repo's structure
    )
    # Now deploy the loaded flow (awaiting the deploy call)
    deployment = await loaded_flow.deploy(
        name="my_deployment",
        parameters={
            "github_repos": [
                "wasit7/papapipeline",
                "wasit7/DjangoCrafter",
                "PrefectHQ/prefect",
                "huggingface/transformers"
            ]
        },
        work_pool_name="default-agent-pool",
        cron="*/5 * * * *"  # Run every 5 minutes (adjust as needed)
    )
    return deployment

# Run the async function in Jupyter using await at top-level
deployment_result = await register_deployment()
print("Deployment registered:", deployment_result)


Looks like you're deploying to a process work pool. If you're creating a deployment for local development, calling 
`.serve` on your flow is a great way to get started. See the documentation for more information: 
https://docs.prefect.io/latest/concepts/flows/#serving-a-flow.  Set `ignore_warnings=True` to suppress this 
message.

/usr/local/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Successfully created/updated all deployments!

                  Deployments                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓
┃ Name                     ┃ Status  ┃ Details ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩
│ show-stars/my_deployment │ applied │         │
└──────────────────────────┴─────────┴─────────┘

To execute flow runs from this deployment, start a worker in a separate terminal that pulls work from the 
'default-agent-pool' work pool:

        $ prefect worker start --pool 'default-agent-pool'

To schedule a run for this deployment, use the following command:

        $ prefect deployment run 'show-stars/my_deployment'

You can also run your flow via the Prefect UI: http://prefect-server:4200/deployments/deployment/8fca78ad-deee-4bbf-8026-8147617b3dec

Deployment registered: 8fca78ad-deee-4bbf-8026-8147617b3dec
